# python_summary_part 6

## OOP inheritance and polymorphism 继承和多态




## Decorator 装饰器
- gives an functionality to a function.
- 装饰器 (Decorator) 是 Python 中的一种高级特性，用来在 不改变原函数或类定义的情况下，动态地为函数、方法或类添加额外功能。  
装饰器本质上是一个函数，它接受一个函数或类作为输入，并返回一个新的函数或类，增强或修改其行为。

In [5]:
from numbers import Number
import re

class Person:
    """base class containing generic methods that are shared by all subclasses,基类包含所有子类共享的通用方法"""
    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    @property
    def age(self) -> int:
        return self._age
    
    @age.setter
    def age(self, value: int) -> None:
        if not isinstance(value, Number):
            raise TypeError(f"Age must be either int or float not {type(value)}")     
        self._age = value

        if not (0 < value < 125):
            raise ValueError(f"Age must between 1 nd 124, not {value}")

    @ property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, value: str) -> None:
        # (stard with letter A-ö + space + A-ö) could have several，字符串必须以字母开头，允许包含一个可选的空格和第二个单词。整个字符串只能包含这些内容，没有多余的字符或空格。re.search() 会返回匹配的结果，如果没有找到匹配的内容，则返回 None，抛出 ValueError
        # value.strip()：移除字符串两端的空格
        if re.search(r"^[A-ö]+(\s[A-ö]+)?$", value.strip()) is None:
            raise ValueError(f"The value {value} is not a valid name")
        
        self._name = value

    def say_hi(self) -> None:
        print(f"{self.name} says hello")

p1 = Person("Jie", 18)
p1


## Inheritance 继承

In [8]:
class Student(Person):
    def __init__(self, name: str, age: int, studies: str) -> None:
        super().__init__(name, age)
        self.studies = studies

try:
    s1 = Student("Anna", 126, "AI")
except ValueError as err:
    print(err)

Age must between 1 nd 124, not 126


In [12]:
try:
    s1 = Student("Anna  HH", 26, "AI")
except ValueError as err:
    print(err)

The value Anna  HH is not a valid name


In [16]:
s2 = Student("Uncle Ben", 70, "Data science")
s2.age, s2.studies

(70, 'Data science')

In [18]:
class Teacher(Person):
    def __init__(self, name: str, age: int, teaches: str) -> None:
        super().__init__(name, age)
        self.teaches = teaches

    def say_hi(self) -> None:
        print(f"Teacher {self.name} teaches {self.teaches}")

t1 = Teacher("Kokchun", 18, "AI")
t1

## Polymorphism 多态性

In [19]:
people = t1, s2
people

(<__main__.Teacher at 0x109509890>, <__main__.Student at 0x109b702d0>)

In [22]:
for person in people:
    person.say_hi()

Teacher Kokchun teaches AI
Uncle Ben says hello


In [23]:
len([1, 2, 3])

3

In [24]:
len("hello")

5

In [25]:
"hello" * 5

'hellohellohellohellohello'

## Operator Overloading  运算符重载

- positional arguments 位置参数
- validation 验证
- generated expression 生成的表达式                 
           
            
- *numbers：表示 __init__ 方法可以接受任意多个参数，这些参数会被收集到一个 元组 (tuple) 中，例如： v = Vector(1, 2, 3.5, 4)
- 遍历 numbers 中的每个元素，对每个 number 进行验证，检查 number 是否是 Number 类型的实例。不是数字类型，就会抛出 TypeError。
- self._numbers = tuple(float(number) for number in numbers)：将 numbers 中的每个元素都转化为浮点数，并保存为一个不可变的元组 (tuple)。

In [26]:
from numbers import Number

class Vector:
    def __init__(self, *numbers: float) -> None:
        for number in numbers:
            if not isinstance(number, Number):
                raise TypeError(f"{number} id not valid element in Vector")
            
        self._numbers = tuple(float(number) for number in numbers)

    @property
    def numbers(self) -> tuple:
        return self._numbers
    
v1 =Vector(2, 3)
v1

In [27]:
v1.numbers

(2.0, 3.0)

In [38]:
from numbers import Number

class Vector:
    def __init__(self, *numbers: float) -> None:
        for number in numbers:
            if not isinstance(number, Number):
                raise TypeError(f"{number} id not valid element in Vector")
            
        self._numbers = tuple(float(number) for number in numbers)

    @property
    def numbers(self) -> tuple:
        return self._numbers
    
    def __repr__(self) -> str:
        return f"Vector{self.numbers}"

    # Operator Overloading 运算符重载
    def __len__(self) -> int:
        return len(self.numbers)

    def __getitem__(self, item: int) -> float:
        return self.numbers[item]

    def __add__(self, other:Vector) -> Vector:
        # validate that other also is a vector and has same lenth  验证其他也是向量并且具有相同的长度 
        numbers = (a+b for a, b in zip(self.numbers, other.numbers)) 
        return Vector(*numbers)
 

v1 = Vector(2, 3)
v1, len(v1), v1[1] 



# 1. 在 Python 中，当你尝试使用 len() 函数获取自定义对象的长度时（比如 len(v1)），这个对象需要实现特殊方法 __len__。如果不实现这个方法，调用 len() 会报错。
# 2. 如果你想要像下标（[]）一样访问对象的元素（例如 v1[0]），你需要实现 __getitem__ 方法。如果没有这个方法，尝试使用下标访问对象时，会抛出 TypeError。
# 3. 当你想运行加法+运算时，需要调用 __add__ 方法。因此，当你执行 v1 + v2 时，Python 会自动调用 v1.__add__(v2) 来处理加法运算。
# self.numbers 和 other.numbers 都是 Vector 对象的元素序列（通常是一个元组或列表）。
# zip(self.numbers, other.numbers) 会将这两个序列中的元素配对，生成一个包含元组的迭代器，每个元组包含 self.numbers 和 other.numbers 中对应位置的元素。
# 假设 v1.numbers = (1, 2) 和 v2.numbers = (2, 3)，那么 zip(self.numbers, other.numbers) 会生成 [(1, 2), (2, 3)]

(Vector(2.0, 3.0), 2, 3.0)

In [42]:
v2 = Vector(2, 3)
v1 + v2


# subtraction 减法, commutative multiplication 交换乘法

Vector(4.0, 6.0)